In [1]:
import argparse
import pandas as pd
import numpy as np
import peddy ### must be in python version 3.7 for peddy to actually work

In [ ]:
def get_args():
    """Incorporating argparse into the code for interchangeable arguments"""
    parser = argparse.ArgumentParser()
    parser.add_argument("--outliers",
        help="input outlier name")
    parser.add_argument("--ped",
        help="input ped file")
	### out will just be the name of my output file... turn up
    parser.add_argument("--out",
        help="outputfile")
    return parser.parse_args()

In [2]:
ped = peddy.Ped('lynchWGS.ped.txt', 'Paternal_ID' == str) ###import peddy file

In [3]:
df = pd.read_table('LaurelSTRsupdated.tsv', delim_whitespace = True, index_col = 0, dtype = {'sample' : str})

In [5]:
def has_parents(sample):
    """Check if Peddy sample has both parents in ped file"""
    if sample.mom is not None and sample.dad is not None:
        return True
    return False

In [6]:
for sample in ped.samples():
        if has_parents(sample):
            print(sample.maternal_id)

209
209
209
987
224
224
224
224
236
198
198
221
207
216
192
193
187
187
187
225
225
225
225


In [7]:
dfkid = df.loc[df['sample'] == '003'] ###match the data frame to the samples of the individual or "kid"
###dfkid['mutation'] = mutation
###add a new column matched by sample mutation from mom and dad
### the above line generates a loc error possibily based on a misunderstanding, but be aware of it
dfmom = df.loc[df['sample'] == '209']
dfdad = df.loc[df['sample'] == '239']

In [8]:
dfmom

,left,right,locus,sample,repeatunit,allele1_est,allele2_est,spanning_reads,spanning_pairs,left_clips,right_clips,unplaced_pairs,sum_str_counts,sum_str_log,depth,outlier,p,p_adj
chrom,,,,,,,,,,,,,,,,,,
chr15,74884270,74884271,chr15-74884270-74884271-T,209,T,0.0,156.34,22.0,28.0,0.0,8.0,0.0,438.0,3.8,32.0,17.01,3.300000e-65,4.400000e-61
chr6,159980183,159980239,chr6-159980183-159980239-AAAG,209,AAAG,0.0,26.59,0.0,19.0,0.0,0.0,0.0,180.0,3.0,23.0,16.27,7.500000e-60,5.100000e-56
chr15,92296076,92296077,chr15-92296076-92296077-AAAG,209,AAAG,0.0,15.86,8.0,34.0,0.0,2.0,0.0,83.0,1.8,24.0,15.17,2.800000e-52,1.200000e-48
chr17,64219424,64219425,chr17-64219424-64219425-AAAAT,209,AAAAT,0.0,15.18,19.0,24.0,0.0,0.0,0.0,107.0,2.2,23.0,14.93,1.100000e-50,3.600000e-47
chr18,24098405,24098406,chr18-24098405-24098406-C,209,C,0.0,84.00,13.0,22.0,0.0,0.0,0.0,142.0,2.5,26.0,14.71,2.800000e-49,7.500000e-46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,0,0,chrX-113126218-113126219-C,209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN
NaN,0,0,chrX-340005-340006-C,209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN
NaN,0,0,chrX-340215-340216-C,209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN


In [9]:
dfkid = dfkid.rename(columns={"allele1_est":"allele1kid", "allele2_est":"allele2kid", "depth": "depth_kid"})
dfdad = dfdad.rename(columns={"allele1_est":"allele1dad", "allele2_est":"allele2dad", "depth": "depth_dad"})
dfmom = dfmom.rename(columns={"allele1_est":"allele1mom", "allele2_est":"allele2mom","depth": "depth_mom"})

In [10]:
drop_from_dkid= ['spanning_reads', 'spanning_pairs', 'left_clips', 'right_clips', 'unplaced_pairs', 'sum_str_counts', 'sum_str_log', 'outlier']
drop_from_parents = ['left', 'right', 'chrom', 'chrom_path', 'right_path', 'left_path', 'disease', 'repeatunit_path', 'overlap', 'sample', 'p', 'p_adj', 'repeatunit'] + drop_from_dkid
not_in_df = []
for item in drop_from_parents:
    if item not in df.columns:
        not_in_df.append(item)

In [11]:
for x in not_in_df:
        drop_from_parents.remove(x)
dfkid = dfkid.drop(drop_from_dkid, axis=1)
dfmom = dfmom.drop(drop_from_parents, axis=1)
dfdad = dfdad.drop(drop_from_parents, axis=1)

In [ ]:
dfmom

In [12]:
kiddad = dfkid.merge(dfdad, on= 'locus',)
kiddadmom = kiddad.merge(dfmom, on= 'locus')

In [ ]:
kiddadmom

In [13]:
wgl = 10

In [34]:
for row in kiddadmom:
    kiddict = {
        "allele1": kiddadmom["allele1kid"],
        "allele2": kiddadmom["allele2kid"]
    }
    momdict = {
        "allele1": kiddadmom["allele1mom"],
        "allele2": kiddadmom["allele2mom"]
    }
    daddict = {
        "allele1": kiddadmom["allele1dad"],
        "allele2": kiddadmom["allele2dad"]
    }
    full_allele_check(wgl,momdict,daddict,kiddict)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
kiddict = {
    "allele1": kiddadmom.iat[1,6]
}

In [ ]:
kiddict["allele1"]

In [35]:
def wiggle(wigglecommand):
        if type(wigglecommand) is int:
            wiggleint = wigglecommand
            return wiggleint
        else:
            type(wigglecommand) is str
            wigglestr = int(wigglecommand)
            return wigglestr

In [38]:
def allele_check(allele1,allele2):
    if (allele1 == 'NaN') & (allele2 == 'NaN'):
        allele1 == 'NaN'
    elif (allele1 != 'NaN') & (allele2 == 'NaN'):
        if allele1 >= 350:
            allele2 = 350
            allele1 = 350
        else:
            allele2 = allele1
    elif (allele1 == 'NaN') & (allele2 != 'NaN'):
        if allele2 >= 350:
            allele1 = 350
            allele2 = 350
        else:
            allele1 = allele2
    elif allele2 >= 350:
        allele2 = 350   
    elif allele1 >= 350:
        allele1 = 350
    else:
        allele1 = allele1
        allele2 = allele2
    return allele1, allele2

In [39]:
allele_check('NaN', 'NaN')

('NaN', 'NaN')

In [29]:
def allele_range(wigglecommand, allele1, allele2):
    if allele1 and allele2 == 'NaN':
      return  
    else:
        tple1 = (allele1 - wiggle(wigglecommand), allele1 + wiggle(wigglecommand))
        tple2 = (allele2 - wiggle(wigglecommand), allele2 + wiggle(wigglecommand))
    return tple1, tple2

    

In [ ]:
allele_range(10,150,150)

In [30]:
def get_allele_ranges(wigglecommand, allele1, allele2):
    a, b = allele_check(allele1,allele2)
    if a and b == 'NaN':
        return
    else:
        x, y =(allele_range(wigglecommand, a, b))
        return x, y
    
    

In [ ]:
get_allele_ranges(10,'NaN','NaN')

In [ ]:
kidallele = [150,70]

In [ ]:
kidalleledict = {
    'allele1': 150, 
    'allele2':  70
}

In [31]:
def check_range(wigglecommand, allele1, allele2, kidallele):
    if get_allele_ranges(wigglecommand, allele1, allele2) is None:
        return None
    else:
        x,y = get_allele_ranges(wigglecommand, allele1, allele2)
        a,b = x
        c,d = y
    if kidallele < 350:
        if a <= kidallele <= b or c <= kidallele <= d:
            return True
        if kidallele > a and b and c and d:
            return 'Amplification'
        else: 
            return 'Deletion'
    elif kidallele >= 350 and (a or b or c or d) >= 350:
        return True    
    elif kidallele > a and b and c and d:
        return 'Amplification'
    else: 
        return 'Deletion'

In [ ]:
check_range(wgl, -70, 100, 50)

In [ ]:
wgl = 10

In [23]:
momalleledict = {
    'allele1': 400,
    'allele2': 300
}

In [24]:
dadalleledict = {
    'allele1': 3000,
    'allele2': 'NaN'
}

In [25]:
kidalleledict = {
    'allele1': 300, 
    'allele2':  150
}

In [ ]:
check_range(wgl, momalleledict['allele1'],momalleledict['allele2'],kidalleledict['allele1'])


In [ ]:
check_range(wgl, momalleledict['allele1'],momalleledict['allele2'],kidalleledict['allele2'])


In [32]:
def full_allele_check(wgl, momalleledict, dadalleledict,kidalleledict):
    if (kidalleledict['allele1'] == 'NaN' and kidalleledict['allele2'] == 'NaN') or (momalleledict['allele1'] == 'NaN' and momalleledict['allele2'] == 'NaN') or (dadalleledict['allele1'] == 'NaN' and dadalleledict['allele2'] == 'NaN'):
        return 'Missing alleles,ignore'
    else:
        if check_range(wgl, momalleledict['allele1'],momalleledict['allele2'],kidalleledict['allele1']) is True:
            if check_range(wgl, dadalleledict['allele1'],dadalleledict['allele2'],kidalleledict['allele2']) is True:
                return 'Full match'
            else:
                return 'MV'
        else: 
            if check_range(wgl, momalleledict['allele1'],momalleledict['allele2'],kidalleledict['allele2']) is True:
                if check_range(wgl, dadalleledict['allele1'],dadalleledict['allele2'],kidalleledict['allele1']) is True:
                    return "Full match"
                else: 
                    return 'MV'
            else: 
                if check_range(wgl, dadalleledict['allele1'],dadalleledict['allele2'],kidalleledict['allele1']) is True:
                    return 'MV'
                else: 
                    if check_range(wgl, dadalleledict['allele1'],dadalleledict['allele2'],kidalleledict['allele2']) is True:
                        return 'MV'
                    else:
                        return 'Double MV, likely error'

In [33]:
full_allele_check(wgl,momalleledict,dadalleledict,kidalleledict)

'MV'

In [ ]:
import pytest